深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的**卷积层**、**池化层**与**循环层**。虽然tf.keras提供了大量常用的层，但有时候我们依然希望自定义层。本节将**介绍如何自定义一个层，从而可以被重复调用**。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [2]:
X = tf.random.uniform((2,20))
X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.4356854 , 0.06403697, 0.5120436 , 0.27091622, 0.29149747,
        0.7806339 , 0.90945125, 0.5262741 , 0.94244814, 0.8918685 ,
        0.4276693 , 0.24441385, 0.3186443 , 0.8793372 , 0.98747015,
        0.65027463, 0.09461153, 0.45071793, 0.74821067, 0.23624885],
       [0.39409208, 0.11369026, 0.3374796 , 0.6187278 , 0.9070437 ,
        0.74034333, 0.62815416, 0.52978075, 0.21594918, 0.60024405,
        0.7450746 , 0.66463804, 0.51156104, 0.1221112 , 0.4085127 ,
        0.84300005, 0.4555844 , 0.33209753, 0.5934856 , 0.50640297]],
      dtype=float32)>

## 4.4.1 custom layer without parameters

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和[“模型构造”]一节中介绍的使用`tf.keras.Model`类构造模型类似。下面的`CenteredLayer`类通过继承`tf.keras.layers.Layer`类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了`call`函数里。这个层里不含模型参数。

In [3]:
class CenteredLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

我们可以实例化这个层，然后做前向计算。

In [4]:
layer = CenteredLayer()
layer(np.array([1,2,3,4,5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2])>

我们也可以用它来构造更复杂的模型。

In [5]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(20))
net.add(CenteredLayer())

Y = net(X)
Y

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[-0.4184435 , -0.52496624,  0.27915132,  0.50444794,  0.02441758,
         0.08515316,  0.15767044, -0.85212696,  0.0466932 ,  0.34518534,
        -0.22098258, -0.7702865 , -0.7344525 ,  0.1747012 , -0.4792117 ,
         0.13290197,  0.5453866 , -0.4254615 ,  0.34001023,  0.57560545],
       [-0.8300566 , -0.6612645 ,  0.01351484,  0.55714893,  0.14477718,
         0.11425453,  0.63892025, -0.217182  ,  0.15977637,  0.7903896 ,
         0.4476206 , -0.37559366, -0.84034616,  0.22046238, -0.6303827 ,
         0.39769125,  0.71456456,  0.0197661 ,  0.21650174,  0.3340444 ]],
      dtype=float32)>

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [6]:
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=2.9802323e-09>

## 4.4.2 custom layer with parameters

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

In [8]:
class myDense(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def build(self, input_shape):  # 这里 input_shape 是第一次运行call()时参数inputs的形状
        self.w = self.add_weight(name='w', shape=[input_shape[-1], self.units],
                                initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b', shape=[self.units], initializer=tf.zeros_initializer())
        
    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

下面，我们实例化`MyDense`类并访问它的模型参数。我们可以直接使用自定义层做前向计算。

In [9]:
dense = myDense(3)
dense(X)
dense.get_weights()

[array([[-0.01456853, -0.00062965, -0.06465482],
        [ 0.0214258 , -0.08342012, -0.00020358],
        [-0.03892386,  0.00263878, -0.05131857],
        [ 0.01586996, -0.02732763,  0.00989991],
        [ 0.00327608,  0.06509433,  0.03225223],
        [ 0.0061471 , -0.03545168, -0.05269927],
        [-0.06773756, -0.01612038,  0.00980218],
        [-0.02087299,  0.01069602, -0.01973275],
        [-0.07711153,  0.01964681, -0.08895659],
        [ 0.00998701, -0.02430547, -0.01208448],
        [-0.08442186,  0.01200019, -0.0936804 ],
        [-0.06103753,  0.05221915, -0.04604558],
        [ 0.00172085, -0.00629787,  0.02116935],
        [ 0.00241339,  0.00293595,  0.02012692],
        [-0.00548004, -0.02211613,  0.02458175],
        [ 0.00367129, -0.05621688,  0.05319601],
        [ 0.04211259,  0.03918486,  0.02719019],
        [ 0.05811013,  0.04471269,  0.01255583],
        [-0.07413621, -0.1553624 , -0.04221198],
        [-0.01060315,  0.09973762,  0.0315074 ]], dtype=float32),
 ar

我们也可以使用自定义层构造模型。

In [10]:
net = tf.keras.models.Sequential()
net.add(myDense(8))
net.add(myDense(1))

net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.01821579],
       [-0.01084319]], dtype=float32)>

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_deep-learning-computation/custom-layer.html)

